In [30]:
# Run with Kernel: gemini-chat-env

In [31]:
# ==== Install & imports ====

import pickle
from typing import List, Dict, Any, Tuple
from pathlib import Path
import numpy as np

# Gemini
from google import genai
from google.genai import types

In [32]:
# ==== Configure paths and models ==== 

# Embedding model used for your stored vectors (3072-D by default).
EMBED_MODEL = "gemini-embedding-001"

# Chat model to answer with context from the best transcript.
CHAT_MODEL = "gemini-2.0-flash-lite"

In [33]:

def read_key_from_env_file(filename: str = "key.env", var_name: str = "GEMINI_API_KEY") -> str:
    p = Path.cwd() / filename  # current working directory (your notebook folder)
    if not p.exists():
        raise FileNotFoundError(f"'{filename}' not found in {Path.cwd()}")
    for line in p.read_text(encoding="utf-8").splitlines():
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        if "=" in line:
            k, v = line.split("=", 1)
            if k.strip() == var_name:
                return v.strip().strip('"').strip("'")
    raise RuntimeError(f"{var_name} not found in {filename}")

API_KEY = read_key_from_env_file()

# Create a single shared client for both embedding + chat.
client = genai.Client(api_key=API_KEY)

In [34]:
# ==== Load data + Validation ====

DATA_FILE = "transcript_embeddings.pkl"

with open(DATA_FILE, "rb") as f:
    records = pickle.load(f)

print(f"Loaded {len(records)} customer records from '{DATA_FILE}'.")

Loaded 50 customer records from 'transcript_embeddings.pkl'.


In [35]:
# ==== Record selection helper ====

def select_customer(records: List[Dict[str, Any]], selector: int) -> Dict[str, Any]:
    """
    Selects a single customer record.

    Behavior:
    - First try exact match on customer_id == selector.
    - If not found, treat 'selector' as 1-based index into the list (1..len(records)).

    Returns the chosen record (dict with 'customer_id', 'transcripts', 'embeddings').
    """
    # Try customer_id match.
    for rec in records:
        if rec["customer_id"] == selector:
            return rec

    # Fallback: 1-based position in the list.
    idx = selector - 1
    assert 0 <= idx < len(records), f"Client index {selector} is out of range."
    return records[idx]

In [36]:
# ==== Embedding & ranking utilities ====

# Normalization is not necesary with the Gemini embedding model.
# However, it´s included to be robust in case of future changes to the model or different embedding models.
def l2_normalize(vecs: np.ndarray) -> np.ndarray:
    """
    L2-normalizes vectors row-wise. Safe even if some rows are already normalized.
    vecs: shape (n, d) or (d,)
    returns: same shape, L2-norm ~ 1.0 per row (or vector)
    """
    vecs = np.asarray(vecs, dtype=np.float32)
    if vecs.ndim == 1:
        denom = np.linalg.norm(vecs) + 1e-12
        return vecs / denom
    denom = np.linalg.norm(vecs, axis=1, keepdims=True) + 1e-12
    return vecs / denom


def embed_prompt(prompt: str) -> np.ndarray:
    """
    Embeds the user prompt using the same Gemini embedding model.
    Returns a (3072,) float32 numpy array.
    """
    # Request a single embedding for the prompt.
    result = client.models.embed_content(
        model=EMBED_MODEL,
        contents=prompt,
        # For RAG, RETRIEVAL_QUERY is a good default:
        config=types.EmbedContentConfig(task_type="RETRIEVAL_QUERY")
    )
    [embedding_obj] = result.embeddings
    v = np.array(embedding_obj.values, dtype=np.float32)

    # gemini-embedding-001 returns normalized vectors at 3072-D by default,
    # but we normalize defensively to be robust if dimensions/settings change later.
    return l2_normalize(v)


def rank_transcripts_by_cosine(query_vec: np.ndarray,
                               doc_vecs: np.ndarray) -> List[Tuple[int, float]]:
    """
    Ranks 4 transcript vectors (doc_vecs shape (4, 3072)) by cosine similarity to query_vec.
    Returns a list of (index, score) sorted descending by score.

    Since vectors are L2-normalized, cosine similarity reduces to a simple dot product.
    """
    q = l2_normalize(query_vec)
    D = l2_normalize(doc_vecs)  # (4, 3072)

    scores = (D @ q)  # (4,)
    order = np.argsort(-scores)  # descending
    return [(int(i), float(scores[i])) for i in order]


In [37]:
# ==== Printing utilities ====

def print_ranked_transcripts(transcripts: List[str],
                             ranking: List[Tuple[int, float]],
                             show_full: bool = True,
                             preview_chars: int = 400) -> None:
    """
    Prints all 4 transcripts in similarity order with scores.
    Set show_full=False to print a shorter preview per transcript.
    """
    print("=" * 100)
    print("Top 4 transcripts by cosine similarity")
    print("=" * 100)
    for rank, (idx, score) in enumerate(ranking, start=1):
        print(f"\n[{rank}] Transcript #{idx+1} — cosine={score:.4f}")
        print("-" * 100)
        text = transcripts[idx]
        if show_full:
            print(text)
        else:
            print(text[:preview_chars] + ("..." if len(text) > preview_chars else ""))


In [38]:
# ==== Main: set inputs (your prompt + client), rank, print, ask Gemini ====

# ==== 1) Provide inputs here ====
USER_PROMPT = "What home service does the customer require?"
CLIENT_SELECTOR = 19  # Example: pick customer_id==17 or 17th row if no matching id.

# ==== 2) Select the customer and collect doc vectors ====
rec = select_customer(records, CLIENT_SELECTOR)
doc_texts: List[str] = rec["transcripts"]
doc_vecs = np.array(rec["embeddings"], dtype=np.float32)  # shape (4, 3072)

# ==== 3) Embed the user prompt and rank the 4 transcripts ====
q_vec = embed_prompt(USER_PROMPT)
ranking = rank_transcripts_by_cosine(q_vec, doc_vecs)

# ==== 4) Print all 4 transcripts in order of similarity ====
print_ranked_transcripts(doc_texts, ranking, show_full=True)


Top 4 transcripts by cosine similarity

[1] Transcript #2 — cosine=0.6494
----------------------------------------------------------------------------------------------------
Hello? Yes, how can I help you? Yes, hi, is this the glass docr you over for [ORGANIZATION] in [LOCATION]? Yes, ma'am. Okay. Just wanted to make sure I had the right phone number because I just did a quick search. I didn't have my glasses on. I have my windshield replace there not too long ago. And the little bracket part that goes around the rearview mirror, you know it's one of those ones as like camera and stuff and you know for you know one of those fancy whatever windshield. And he said something about when I. When he did, he said the brackets were being troublesome but the [MEDICAL_CONDITION] pieces keeps. It keeps falling and it won't stay up anymore. And I was just curious if that's something that y'all could take a look at and see if there was something that needed to be fixed on it. Yeah, let me see. I h

In [39]:
# ==== Build a grounded prompt from the top transcript and call Gemini chat ====

best_idx = ranking[0][0]
best_transcript = doc_texts[best_idx]

# Keep the chat input simple and explicit:
chat_input = f"""
You are assisting an internal call-center analysis workflow.

User prompt:
{USER_PROMPT}

Relevant transcript (the most semantically similar of 4 for this customer):
{best_transcript}

Guidelines:
- Rely primarily on the transcript content to answer the user prompt.
- If the transcript is missing details the prompt asks for, state what is missing.
- Keep the answer crisp and actionable.
"""

response = client.models.generate_content(
    model=CHAT_MODEL,
    contents=chat_input
)

print("\n" + "="*100)
print("Gemini answer (using the top-matching transcript as context):")
print("="*100 + "\n")
print(response.text)


Gemini answer (using the top-matching transcript as context):

The customer requires service to fix the bracket part around their rearview mirror that is falling down.

